In [11]:
import pandas as pd
import plotly.graph_objects as go
from datetime import time
import re

# Load your Excel data
file_path = 'Todays Python Workflow.xlsx'
df = pd.read_excel(file_path)

# Filter out rows where Qty equals 0
df = df[df['Qty'] != 0]

# Ensure the Premium is calculated correctly (Qty * Price * 100)
df['Premium'] = df['Qty'] * df['Price'] * 100

# Clean up Bot/Sold column by stripping extra spaces and converting to uppercase using .loc[]
df.loc[:, 'Bot/Sold'] = df['Bot/Sold'].str.strip().str.upper()

# Separate the data for graph (excluding 'MID')
df_graph = df[df['Bot/Sold'].isin(['BOT', 'SOLD'])].copy()

# Extract only the time part if the time is part of a datetime string
df_graph['Time_only'] = df_graph['Time'].astype(str).str.slice(-8)  # Assumes time is the last 8 characters (HH:MM:SS)

# Convert the sliced string to time format
df_graph['Time_only'] = pd.to_datetime(df_graph['Time_only'], format='%H:%M:%S', errors='coerce').dt.time

# Check if there are any parsing errors (NaT values)
df_graph = df_graph.dropna(subset=['Time_only'])

# Classify trades as Bot Call, Sold Call, Bot Put, and Sold Put
df_graph['Bot Call'] = df_graph.apply(lambda row: row['Premium'] if row['Type'] == 'Call' and row['Bot/Sold'] == 'BOT' else 0, axis=1)
df_graph['Sold Call'] = df_graph.apply(lambda row: row['Premium'] if row['Type'] == 'Call' and row['Bot/Sold'] == 'SOLD' else 0, axis=1)
df_graph['Bot Put'] = df_graph.apply(lambda row: row['Premium'] if row['Type'] == 'Put' and row['Bot/Sold'] == 'BOT' else 0, axis=1)
df_graph['Sold Put'] = df_graph.apply(lambda row: row['Premium'] if row['Type'] == 'Put' and row['Bot/Sold'] == 'SOLD' else 0, axis=1)

# Extract the strike price (4-digit number after the date) from the 'Option' column for sorting purposes
def extract_strike(option_str):
    # Use regex to capture the last 4-digit number (strike price) in the option name
    match = re.search(r'(\d{4})\s*[CP]', option_str)
    if match:
        return int(match.group(1))
    return 0  # Default return if no match is found

df_graph['Strike Value'] = df_graph['Option'].apply(extract_strike)

# Sort by 'Strike Value' and 'Type' (to ensure Call/Put is sorted alongside)
df_graph = df_graph.sort_values(by=['Strike Value', 'Type'])

# Function to create and show bar charts for individual transactions
def create_transaction_bar_chart(df_segment, title):
    fig = go.Figure()

    # Now that data is sorted, the x-axis will reflect sorted strike prices (both Calls and Puts)
    for transaction_type, color in zip(['Bot Call', 'Sold Call', 'Bot Put', 'Sold Put'], 
                                       ['blue', 'lightcoral', 'red', 'lightblue']):
        # Select transactions where the current transaction type is not zero
        filtered_data = df_segment[df_segment[transaction_type] != 0].copy()
        
        # For each transaction, create a bar segment for the total premium
        for index, row in filtered_data.iterrows():
            fig.add_trace(go.Bar(
                x=[row['Option']], 
                y=[row[transaction_type]],  # Single transaction premium
                name=transaction_type, 
                marker_color=color,
                hovertemplate=f'Premium: {row["Premium"]}<br>Time: {row["Time_only"]}',  # Custom hover data
                showlegend=False
            ))

    # Update the layout to use stacked bars
    fig.update_layout(
        title=title,
        xaxis_title='Option (Strike Price or Identifier)',
        yaxis_title='Premium Amount',
        barmode='stack',  # Keep the bars stacked
        template='plotly_dark'
    )
    
    fig.show()

# Function to filter, group, and generate top 20 options for a given time segment
def filter_and_create_top_20_chart(df_graph, start_time, end_time, segment_title):
    # Filter for the time segment
    df_segment = df_graph[(df_graph['Time_only'] >= start_time) & (df_graph['Time_only'] < end_time)]

    # Group by Option and calculate total premium for the time segment
    df_grouped_segment = df_segment.groupby('Option').agg({'Premium': 'sum'}).reset_index()

    # Sort by total premium and select the top 20 options
    top_20_options_segment = df_grouped_segment.sort_values(by='Premium', ascending=False).head(20)['Option']

    # Filter the original data to include only the top 20 options for this segment
    df_segment_top_20 = df_segment[df_segment['Option'].isin(top_20_options_segment)]

    # Create the bar chart for this time segment
    create_transaction_bar_chart(df_segment_top_20, segment_title)

# Define time segments
time_segments = [
    (time(5, 0, 0), time(6, 0, 0), 'Top 20 Premium by Unique Option (05:00:00-06:00:00)'),
    (time(6, 0, 0), time(7, 0, 0), 'Top 20 Premium by Unique Option (06:00:00-07:00:00)'),
    (time(7, 0, 0), time(8, 0, 0), 'Top 20 Premium by Unique Option (07:00:00-08:00:00)'),
    (time(8, 0, 0), time(9, 0, 0), 'Top 20 Premium by Unique Option (08:00:00-09:00:00)'),
    (time(9, 0, 0), time(10, 0, 0), 'Top 20 Premium by Unique Option (09:00:00-10:00:00)'),
    (time(10, 0, 0), time(11, 0, 0), 'Top 20 Premium by Unique Option (10:00:00-11:00:00)'),
    (time(11, 0, 0), time(12, 0, 0), 'Top 20 Premium by Unique Option (11:00:00-12:00:00)'),
    (time(12, 0, 0), time(13, 0, 0), 'Top 20 Premium by Unique Option (12:00:00-13:00:00)'),
    (time(13, 0, 0), time(14, 0, 0), 'Top 20 Premium by Unique Option (13:00:00-14:00:00)')
]

# Loop through each time segment and generate the charts
for start_time, end_time, segment_title in time_segments:
    filter_and_create_top_20_chart(df_graph, start_time, end_time, segment_title)
